Follow this tutorial
https://notebook.community/biosustain/cameo-notebooks/other/co-factor-swapping


In [1]:
from cameo import models
model_orig = models.bigg.iJO1366

KeyboardInterrupt: 

In [ ]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [3]:
model = model_orig.copy()


In [5]:
for rid in ['FHL', 'CAT', 'SPODM', 'SPODMpp']:
    model.reactions.get_by_id(rid).knock_out()
model.reactions.POR5.lower_bound = 0
model.reactions.EX_glc__D_e.lower_bound = -10
model.reactions.EX_o2_e.lower_bound = -10

In [6]:
model.reactions.BIOMASS_Ec_iJO1366_core_53p95M.lower_bound = 0.1


In [8]:
model.objective = model.reactions.EX_thr__L_e
#b(model.solve().f * 4) / (model.reactions.EX_glc__D_e.flux * 6)

In [34]:
py = product_yield(model.reactions.EX_thr__L_e, model.reactions.EX_glc__D_e)
optswap = CofactorSwapOptimization(model=model, objective_function=py)
optswap2 = biomass_coupled_product_yield(model=model, objective_function=py)

NameError: name 'biomass_coupled_product_yield' is not defined

In [13]:
saved_swapped_reactions = optswap.run(max_evaluations=2000, max_size=2)


Starting optimization at Thu, 17 Nov 2022 17:27:28


HBox()

Finished after 00:01:20


In [15]:
saved_swapped_reactions

,index,targets,fitness
0,0,"(TRSARr,)",0.805496
1,2,"(GAPD, GLUDy)",0.804368
2,3,"(PDH, GAPD)",0.799077
3,4,"(PGCD, GAPD)",0.788334
4,5,"(GAPD, IPMD)",0.788167
5,6,"(GAPD, KARA2)",0.788146
6,7,"(GTHOr, GAPD)",0.788141
7,8,"(P5CR, GAPD)",0.788137
8,9,"(GLYCL, GAPD)",0.788115
9,10,"(GAPD,)",0.788108


In [17]:
list(saved_swapped_reactions)[0:10]

[[('TRSARr',), 0.805496],
 [('GAPD', 'GLUDy'), 0.804368],
 [('PDH', 'GAPD'), 0.799077],
 [('PGCD', 'GAPD'), 0.788334],
 [('GAPD', 'IPMD'), 0.788167],
 [('GAPD', 'KARA2'), 0.788146],
 [('GTHOr', 'GAPD'), 0.788141],
 [('P5CR', 'GAPD'), 0.788137],
 [('GLYCL', 'GAPD'), 0.788115],
 [('GAPD',), 0.788108]]

In [24]:
optswap.model.objective = optswap.model.reactions.EX_thr__L_e

In [26]:
optswap.model

Name,iJO1366
Memory address,0x07fa4466e5160
Number of metabolites,1805
Number of reactions,2583
Number of groups,0
Objective expression,1.0*EX_thr__L_e - 1.0*EX_thr__L_e_reverse_ddaf9
Compartments,"cytosol, extracellular space, periplasm"


In [27]:
from sympy.solvers import solve

In [28]:
original = (optswap.model.solve().f * 4) / (-optswap.model.reactions.EX_glc__D_e.flux * 6)

AttributeError: 'Model' object has no attribute 'solve'

In [18]:
optswap.model.reactions.EX_thr__L_e.model = optswap.model
optswap.model.objective = optswap.model.reactions.EX_thr__L_e
original = (optswap.model.solve().f * 4) / (-optswap.model.reactions.EX_glc__D_e.flux * 6)
with TimeMachine() as tm:
    optswap.model.swap_reaction('GAPD', tm)
    swapped = (optswap.model.solve().f * 4) / (-optswap.model.reactions.EX_glc__D_e.flux * 6)

print("product/substrate yield without swap: {}\nproduct/substrate yield with swap: {}".format(original, swapped))

AttributeError: can't set attribute

In [30]:
reactions=saved_swapped_reactions.data_frame.targets


In [31]:
#Fra Marks kode:b


for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()

Reactions:
2-Hydroxy-3-oxopropanoate + H+ + Nicotinamide adenine dinucleotide - reduced ==> (R)-Glycerate + Nicotinamide adenine dinucleotide

Reactions:
Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

L-Glutamate + H2O H2O + Nicotinamide adenine dinucleotide phosphate ==> 2-Oxoglutarate + H+ + Ammonium + Nicotinamide adenine dinucleotide phosphate - reduced

Reactions:
Coenzyme A + Pyruvate + Nicotinamide adenine dinucleotide ==> Acetyl-CoA + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced

Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucleotide ==> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
3-Phospho-D-glycerate + Nicotinamide adenine dinucleotide ==> 3-Phosphohydroxypyruvate + H+ + Nicotinamide adenine dinucleotide - reduced

Glyceraldehyde 3-phosphate + Phosphate + Nicotinamide adenine dinucle

# Theoretical maximal yield
From Marks report

In [33]:
with model:
    model.objective = model.reactions.3OAR80 #Setting the objective to the reaction, which is producing Lycopene
    lycopene_production = model.optimize().objective_value
    print('Maximum productivity of lycopene =', lycopene_production, 'mmol/gDW*h')

    glc_uptake_flux = model.reactions.r_1714.flux 
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc')

SyntaxError: invalid syntax (2811274944.py, line 2)

In [ ]:
model.reactions.R